# PYTHON LIBRARIES

In [92]:
# dash Libraries
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# plotly Libraries
import plotly.graph_objects as go
import dash_table

# pandas Libraries
import pandas as pd


# keras Library
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# numpy Library
import numpy as np

DATA CLEANING AND SCALING

In [93]:
# dataset reading
df_nse = pd.read_csv("data/NSE-Tata.csv")
df = pd.read_csv("data/stock_data.csv")

In [94]:
# index formatting
df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date']

In [95]:
# data sorting
data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])

In [96]:
for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]

new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)

dataset=new_data.values

Dataset split to Train and Test

In [97]:
# 
train=dataset[0:987,:]
valid=dataset[987:,:]

Scaling of data

In [98]:

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

In [99]:
x_train, y_train = [],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

In [100]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

MODEL LOADING

In [101]:
model = load_model("data/saved_lstm_model.h5")

In [102]:
inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

In [103]:
X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

8/8 [==============================] - 1s 9ms/step


DATASET SPLIT TO VALIDATION AND TRAINING

In [104]:
train=new_data[:987]
valid=new_data[987:]
# valid['Predictions']=closing_price

In [105]:
valid.loc[:,['Predictions']] = closing_price

In [106]:
valid.head()

,Close,Predictions
Date,,
2017-10-09,208.3,201.261261
2017-10-10,208.45,202.065033
2017-10-11,209.4,202.764038
2017-10-12,212.0,203.434250
2017-10-13,210.25,204.296951


REGREESION METRICS FOR MODEL

In [107]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [108]:
mae = mean_absolute_error(valid["Close"], valid["Predictions"])
print("mean_absolute_error",mae)


mse = mean_squared_error(valid["Close"], valid["Predictions"])
print("mean_squared_error",mse)


rmse = np.sqrt(mean_squared_error(valid["Close"], valid["Predictions"]))
print("root_mean_squared_error",rmse)

r2 = r2_score(valid["Close"], valid["Predictions"])
print("r-squared_score",r2)

mean_absolute_error 7.912116758285032
mean_squared_error 112.79968733858435
root_mean_squared_error 10.620719718483505
r-squared_score 0.8611431050644682


In [109]:
regression_metrics = {
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2
}

# DASHBOARD

In [110]:
app = dash.Dash(__name__)
server = app.server

In [118]:
app.layout = html.Div([
   
    html.H1("Stock Price Analysis Dashboard"),
   
    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='NSE-TATAGLOBAL Stock Data', className='custom-tab', children=[
			html.Div([
				html.H2("Actual closing price", className='custom-content'),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=train.index,
								y=train["Close"],
								mode='lines'
							)

						],
						"layout":go.Layout(
							title='Actual Closing Rate(2013-2014)',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}
				),
				html.H2("LSTM Predicted closing price", className='custom-content'),
				dcc.Graph(
					id="Predicted Data",
					figure={
						"data":[
							go.Scatter(
								x=valid.index,
								y=valid["Predictions"],
								mode='lines'
							)
						],
						"layout":go.Layout(
							title='Predicted Closing Rate(2017-2018)',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}
				),
                html.H2("Model Metrics Table", className='custom-content'),
                dash_table.DataTable(
        			id='metrics-table',
        			columns=[
        			    {'name': 'Metric', 'id': 'Metric'},
        			    {'name': 'Value', 'id': 'Value'}
        			],
        			data=[{'Metric': metric, 'Value': value} for metric, value in regression_metrics.items()])

			])        		
        ]),


        dcc.Tab(label='NSE-TATAGLOBAL Analysis',className='custom-tab', children=[
            html.Div([
                html.H2("Tata Stock Actual vs Predicted closing price", className='custom-content'),

                dcc.Dropdown(id= 'my-dropdown3',
                             options=[{'label': 'Actual Closing', 'value': 'Close'},
                                      {'label': 'Predicted Closing', 'value': 'Predictions'}],
                            multi=True,value=['Close'], className='dropdown-menu'),
                dcc.Graph(id='closing')
            ])
        ]),
        

        dcc.Tab(label='Analyse',className='custom-tab', children=[
            html.Div([
                html.H2("Stocks High vs Lows", className='custom-content'),
              
                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'}, 
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'], className='dropdown-menu'),
                dcc.Graph(id='highlow'),


                html.H2("Stocks Market Volume", className='custom-content'),
         
                dcc.Dropdown(id='my-dropdown2',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'},
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'],className='dropdown-menu'),
                dcc.Graph(id='volume')
            ], className="container"),
        ])


    ])
])

In [112]:
# app.layout = html.Div([
#     html.H1("Stock Price Analysis Dashboard"),
#     dcc.Tabs(
#         id="tabs",
#         value='tab-1',
#         children=[
#             dcc.Tab(label='Model and Stock Summary', value='tab-1', className='custom-tab'),
#             dcc.Tab(label='NSE-TATAGLOBAL Analysis', value='tab-2', className='custom-tab'),
#             dcc.Tab(label='Stock Analysis', value='tab-3', className='custom-tab')
#         ],
#         # colors={
#         #     "border": "#ba181b",
#         #     "primary": "#e5383b",
#         #     "background": "#b1a7a6"
#         # }
#     ),
#     html.Div(id='tabs-content')
# ])

In [113]:
# @app.callback(Output('tabs-content', 'children'),
#               Input('tabs', 'value'))
# def render_content(tab):
#     if tab == 'tab-1':
#         return html.Div([
#             # Add content for Tab 1 here
#             html.H3('Actual closing price', className='custom-content'),
#                 dcc.Graph(
# 					id="Actual Data",
# 					figure={
# 						"data":[
# 							go.Scatter(
# 								x=train.index,
# 								y=train["Close"],
# 								mode='lines'
# 							)

# 						],
# 						"layout":go.Layout(
# 							title='Actual Closing Rate(2013-2014)',
# 							xaxis={'title':'Date'},
# 							yaxis={'title':'Closing Rate'}
# 						)
# 					}
# 				),
#             html.H3("LSTM Predicted closing price",className='custom-content'),
# 				dcc.Graph(
# 					id="Predicted Data",
# 					figure={
# 						"data":[
# 							go.Scatter(
# 								x=valid.index,
# 								y=valid["Predictions"],
# 								mode='lines'
# 							)
# 						],
# 						"layout":go.Layout(
# 							title='Predicted Closing Rate(2017-2018)',
# 							xaxis={'title':'Date'},
# 							yaxis={'title':'Closing Rate'}
# 						)
# 					}
# 				)				
            
#         ])
#     elif tab == 'tab-2':
#         return html.Div([
#             # Add content for Tab 2 here
#             html.H3('Tata Stock Actual vs Predicted closing price', className='custom-content'),
#             dcc.Dropdown(id= 'my-dropdown3',
#                              options=[{'label': 'Actual Closing', 'value': 'Close'},
#                                       {'label': 'Predicted Closing', 'value': 'Predictions'}],
#                             multi=True,value=['Close'],
#                             style={"display": "block", "margin-left": "auto", 
#                                     "margin-right": "auto", "width": "60%"}),
#                 dcc.Graph(id='closing')
#         ])
#     elif tab == 'tab-3':
#         return html.Div([
#             html.H3('Stocks High vs Lows', className='custom-content'),
#             # Add content for Tab 3 here
# 			dcc.Dropdown(id='my-dropdown',
#                              options=[{'label': 'Tesla', 'value': 'TSLA'},
#                                       {'label': 'Apple','value': 'AAPL'}, 
#                                       {'label': 'Facebook', 'value': 'FB'}, 
#                                       {'label': 'Microsoft','value': 'MSFT'}], 
#                              multi=True,value=['FB'],
#                              style={"display": "block", "margin-left": "auto", 
#                                     "margin-right": "auto", "width": "60%"}),
#                 dcc.Graph(id='highlow'),
            
#             html.H3('Stocks Market Volume', className='custom-content'),
#             dcc.Dropdown(id='my-dropdown2',
#                              options=[{'label': 'Tesla', 'value': 'TSLA'},
#                                       {'label': 'Apple','value': 'AAPL'}, 
#                                       {'label': 'Facebook', 'value': 'FB'},
#                                       {'label': 'Microsoft','value': 'MSFT'}], 
#                              multi=True,value=['FB'],
#                              style={"display": "block", "margin-left": "auto", 
#                                     "margin-right": "auto", "width": "60%"}),
#                 dcc.Graph(id='volume')
#         ],className="container"),

In [114]:
@app.callback(Output('highlow', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph_highlow(selected_dropdown):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    trace2 = []
    for stock in selected_dropdown:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["High"],
                     mode='lines', opacity=0.7, 
                     name=f'High {dropdown[stock]}',textposition='bottom center'))
        trace2.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Low"],
                     mode='lines', opacity=0.6,
                     name=f'Low {dropdown[stock]}',textposition='bottom center'))
    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Price (USD)"})}
    return figure

In [115]:
@app.callback(Output('closing','figure'),
              [Input('my-dropdown3', 'value')])
def update_graph_closing(selected_dropdown_close):
    dropdown= {"Close": "Actual","Predictions": "Predicted"}
    trace1=[]
    for stock in selected_dropdown_close:
        trace1.append(
            go.Scatter(x=valid.index,
                       y=valid[stock],
                       mode = 'lines', opacity=0.7,
                       name=f'{dropdown[stock]} Closing ', textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"{''.join(str(dropdown[i]) for i in selected_dropdown_close)} Prices Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Cosing Price"})}
    return figure

In [116]:
@app.callback(Output('volume', 'figure'),
              [Input('my-dropdown2', 'value')])
def update_graph_volume(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    for stock in selected_dropdown_value:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Volume"],
                     mode='lines', opacity=0.7,
                     name=f'Volume {dropdown[stock]}', textposition='bottom center'))
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"Market Volume for {', '.join(str(dropdown[i]) for i in selected_dropdown_value)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Transactions Volume"})}
    return figure

In [117]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)